# database 연결

In [1]:
import pymysql
import pandas as pd
import re
conn = pymysql.connect(host='210.121.218.5', user='zabbix', passwd='zabbix', port=3306, db='test', charset='euckr')
curs = conn.cursor()

## neoss 데이터에 hostid 추가

In [2]:
sql = "SELECT DISTINCT n.*, h.hostid \
FROM ZABBIXDB.neoss AS n, ZABBIXDB.hosts AS h \
WHERE n.leased_line_num IS NOT NULL AND n.interface IS NOT NULL AND n.host_name IS NOT NULL \
AND n.host_name = h.host"
curs.execute(sql)
rows = curs.fetchall()
df = pd.DataFrame(rows)

In [3]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10
0,강남서부법인고객본부,020000000002,21263730000,안산,서안산,Ansan-GDE065-2,TenGi0/14,61.78.44.81,14.51.254.81,221.153.112.64/26,10698
1,강남서부법인고객본부,020000000010,24764882670,양재,서초,Seocho-EXA176,vlan1.77,61.78.42.176,1.1.1.1,None,10640
2,강남서부법인고객본부,020000000024,20363402260,강서,영등포,Gangseo-GDE253-4,TenGi0/6,61.78.43.97,59.15.160.212,222.108.26.0/24,10581
3,강남서부법인고객본부,020000000025,20363558880,강서,영등포,Gangseo-GDE253-4,TenGi0/7,61.78.43.97,59.15.160.213,220.118.218.128/26,10581
4,강남서부법인고객본부,020000000027,20502719980,강서,목동,Gangseo-GDE125-5,TenGi0/7,61.78.43.97,59.15.161.21,221.151.182.128/25,10582


## neoss leased_line_num 기준 Itemid 검색하여 csv 파일 생성

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11766 entries, 0 to 11765
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       11766 non-null  object
 1   1       11766 non-null  object
 2   2       11766 non-null  object
 3   3       11766 non-null  object
 4   4       11766 non-null  object
 5   5       11766 non-null  object
 6   6       11766 non-null  object
 7   7       11498 non-null  object
 8   8       11766 non-null  object
 9   9       11073 non-null  object
 10  10      11766 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 1011.3+ KB


In [5]:
# db_moham_row = []
count = 0
li = [('leased_line_num', 'interface', 'hostname', 'service_id', 'itemid', 'key_', 'hostid', 'in_out', 'regexp')]
li_df = pd.DataFrame(li, columns = ['leased_line_num', 'interface', 'hostname', 'service_id', 'itemid', 'key_', 'hostid', 'in_out',  'regexp'])
for rows in df.iterrows():
    count = count + 1

    interface = rows[1][6]
    hostid = rows[1][10]
    hostname = rows[1][5]
    leased_line_num = rows[1][1]
    service_id = rows[1][2]
    globals()["regexp_list_{}".format(interface)] = []
    # 1. 대다수의 일반적인 interface
    globals()["regexp_list_{}".format(interface)].append('.+(' + interface + ')[^0-9]')
    # 2. interface 에 ifHC~ 포함된 경우
    if ('ifHCInOctets' in interface) or ('ifHCOutOctets' in interface):
        globals()["regexp_list_{}".format(interface)].append('.+(' + interface[interface.find('[')+1:-1] + ')[^0-9]')
    # 3. 'ge3/1/19, IPinterface' 에서 숫자를 추출해야 하는 경우
    try:
        p = re.compile('[0-9]+(/)[0-9]+(/)[0-9]+')
        interface_num = p.search(interface).group()
        globals()["regexp_list_{}".format(interface)].append('.+(' + interface_num + ')[^0-9]')
    except AttributeError:
        pass
#         print('regexp_list: {}'.format( globals()["regexp_list_{}".format(interface)]))
    for index, regexp in enumerate(globals()["regexp_list_{}".format(interface)]):
        sql_command = "SELECT DISTINCT i.itemid, i.key_, i.hostid FROM ZABBIXDB.items as i WHERE \
        i.key_ REGEXP '{}' AND i.hostid = '{}'".format(regexp, hostid)
#             print(sql_command)
        curs.execute(sql_command)
        fet = curs.fetchall()
        if fet == () :
            pass
        else:
            li_df = li_df.append({'leased_line_num' :leased_line_num , 'interface': interface, 'hostname': hostname, 'service_id' : service_id,
                                  'itemid' : fet[0][0], 'key_' : fet[0][1], 'hostid' : fet[0][2], 'in_out': 'in' , 'regexp' :  regexp}, ignore_index = True)
            li_df = li_df.append({'leased_line_num' :leased_line_num , 'interface': interface, 'hostname': hostname, 'service_id' : service_id,
                                  'itemid' : fet[1][0], 'key_' : fet[1][1], 'hostid' : fet[1][2], 'in_out': 'out' , 'regexp' :  regexp}, ignore_index = True)
            break
            #             dic[line_no].append()
    #         print(dic)
    #             db_moham_row.append(curs.fetchall())
#     #             print(db_moham_row)
#             print(count)
        

In [6]:
li_df

,leased_line_num,interface,hostname,service_id,itemid,key_,hostid,in_out,regexp
0,leased_line_num,interface,hostname,service_id,itemid,key_,hostid,in_out,regexp
1,020000000002,TenGi0/14,Ansan-GDE065-2,21263730000,117443,ifHCInOctets[TenGi0/14],10698,in,.+(TenGi0/14)[^0-9]
2,020000000002,TenGi0/14,Ansan-GDE065-2,21263730000,117491,ifHCOutOctets[TenGi0/14],10698,out,.+(TenGi0/14)[^0-9]
3,020000000024,TenGi0/6,Gangseo-GDE253-4,20363402260,117148,ifHCInOctets[TenGi0/6],10581,in,.+(TenGi0/6)[^0-9]
4,020000000024,TenGi0/6,Gangseo-GDE253-4,20363402260,117196,ifHCOutOctets[TenGi0/6],10581,out,.+(TenGi0/6)[^0-9]
...,...,...,...,...,...,...,...,...,...
23062,028884230079,"5/1/8, 10/100/Gig Ethernet SFP",Songdo-EXA020,10196726780,169002,"ifHCOutOctets[""5/1/8, 10/100/Gig Ethernet SFP,...",10622,out,".+(5/1/8, 10/100/Gig Ethernet SFP)[^0-9]"
23063,028884230079,"5/1/8, 10/100/Gig Ethernet SFP",Songdo-EXA020,10196726780,168843,"ifHCInOctets[""5/1/8, 10/100/Gig Ethernet SFP, ...",10622,in,".+(5/1/8, 10/100/Gig Ethernet SFP)[^0-9]"
23064,028884230079,"5/1/8, 10/100/Gig Ethernet SFP",Songdo-EXA020,10196726780,169002,"ifHCOutOctets[""5/1/8, 10/100/Gig Ethernet SFP,...",10622,out,".+(5/1/8, 10/100/Gig Ethernet SFP)[^0-9]"
23065,021901020250,Vlan83,Singal-GDE244-3,21580033120,191973,ifHCInOctets[Vlan83],10467,in,.+(Vlan83)[^0-9]


In [7]:
# 중복제거
li_df = li_df.drop_duplicates(subset=None, keep='first', inplace=False, ignore_index=False)
len(li_df)

16079

## ingre, engre, gbic 열 추가

In [8]:
new_df = li_df.drop(li_df.index[0])
new_df["ingre"] = ""
new_df["engre"] = ""
new_df["gbic"] = ""
len(new_df)

16078

In [9]:
new_df.head()

,leased_line_num,interface,hostname,service_id,itemid,key_,hostid,in_out,regexp,ingre,engre,gbic
1,020000000002,TenGi0/14,Ansan-GDE065-2,21263730000,117443,ifHCInOctets[TenGi0/14],10698,in,.+(TenGi0/14)[^0-9],,,
2,020000000002,TenGi0/14,Ansan-GDE065-2,21263730000,117491,ifHCOutOctets[TenGi0/14],10698,out,.+(TenGi0/14)[^0-9],,,
3,020000000024,TenGi0/6,Gangseo-GDE253-4,20363402260,117148,ifHCInOctets[TenGi0/6],10581,in,.+(TenGi0/6)[^0-9],,,
4,020000000024,TenGi0/6,Gangseo-GDE253-4,20363402260,117196,ifHCOutOctets[TenGi0/6],10581,out,.+(TenGi0/6)[^0-9],,,
5,020000000025,TenGi0/7,Gangseo-GDE253-4,20363558880,117149,ifHCInOctets[TenGi0/7],10581,in,.+(TenGi0/7)[^0-9],,,


## ingre, engre, gbic 채워넣기


In [10]:
curs.close()
conn = pymysql.connect(host='210.121.218.5', user='zabbix', passwd='zabbix', port=3306, db='test', charset='euckr')
curs = conn.cursor()
for index, rows in new_df.iterrows():
    sql_command = "SELECT DISTINCT ingre, engre, gbic FROM ZABBIXDB.realspeed WHERE leased_line_num = '{}'".format(rows[0])
    curs.execute(sql_command)
    fet = curs.fetchall()
    if len(fet) == 0:
        pass
    else :
        new_df.loc[index]['ingre'] = fet[0][0]
        new_df.loc[index]['engre'] = fet[0][1]
        new_df.loc[index]['gbic'] = fet[0][2]

In [11]:
len(new_df)

16078

## bidw contract_speed, service_name 가져오기

In [12]:
curs.close()
conn = pymysql.connect(host='210.121.218.5', user='zabbix', passwd='zabbix', port=3306, db='test', charset='euckr')
curs = conn.cursor()
last_df = new_df
last_df["contract_speed"] = ""
last_df["service_name"] = ""

In [13]:
last_df.head()

,leased_line_num,interface,hostname,service_id,itemid,key_,hostid,in_out,regexp,ingre,engre,gbic,contract_speed,service_name
1,020000000002,TenGi0/14,Ansan-GDE065-2,21263730000,117443,ifHCInOctets[TenGi0/14],10698,in,.+(TenGi0/14)[^0-9],1000000000,1000000000,1000000000,,
2,020000000002,TenGi0/14,Ansan-GDE065-2,21263730000,117491,ifHCOutOctets[TenGi0/14],10698,out,.+(TenGi0/14)[^0-9],1000000000,1000000000,1000000000,,
3,020000000024,TenGi0/6,Gangseo-GDE253-4,20363402260,117148,ifHCInOctets[TenGi0/6],10581,in,.+(TenGi0/6)[^0-9],1000000000,1000000000,1000000000,,
4,020000000024,TenGi0/6,Gangseo-GDE253-4,20363402260,117196,ifHCOutOctets[TenGi0/6],10581,out,.+(TenGi0/6)[^0-9],1000000000,1000000000,1000000000,,
5,020000000025,TenGi0/7,Gangseo-GDE253-4,20363558880,117149,ifHCInOctets[TenGi0/7],10581,in,.+(TenGi0/7)[^0-9],1000000000,1000000000,1000000000,,


In [14]:
for index, rows in last_df.iterrows():
    sql_command = "SELECT DISTINCT service_name, contract_speed FROM ZABBIXDB.bidw WHERE service_id = '{}'".format(rows[3])
    curs.execute(sql_command)
    fet = curs.fetchall()
    if len(fet) == 0:
        pass
    else :
        fet[0][0]
        last_df.loc[index]['contract_speed'] = fet[0][0]
        last_df.loc[index]['service_name'] = fet[0][1]

## new_db 비우기

In [15]:
curs.close()
conn = pymysql.connect(host='210.121.218.5', user='zabbix', passwd='zabbix', port=3306, db='test', charset='euckr')
curs = conn.cursor()
sql_command = "SELECT * FROM ZABBIXDB.new_db"
curs.execute(sql_command)
fet = curs.fetchall()
len(fet)
if fet != 0:
    sql_command = "DELETE FROM ZABBIXDB.new_db"
    curs.execute(sql_command)
    conn.commit()

## key_ 제거하기

In [16]:
drop_df = last_df.drop(['key_'], axis =1)
drop_df = drop_df.drop(['regexp'], axis =1)

## drop_df 를 new_db에 insert 하기

In [17]:
drop_df.head()

,leased_line_num,interface,hostname,service_id,itemid,hostid,in_out,ingre,engre,gbic,contract_speed,service_name
1,020000000002,TenGi0/14,Ansan-GDE065-2,21263730000,117443,10698,in,1000000000,1000000000,1000000000,GiGA office Standard,1G
2,020000000002,TenGi0/14,Ansan-GDE065-2,21263730000,117491,10698,out,1000000000,1000000000,1000000000,GiGA office Standard,1G
3,020000000024,TenGi0/6,Gangseo-GDE253-4,20363402260,117148,10581,in,1000000000,1000000000,1000000000,GiGA office Standard,1G
4,020000000024,TenGi0/6,Gangseo-GDE253-4,20363402260,117196,10581,out,1000000000,1000000000,1000000000,GiGA office Standard,1G
5,020000000025,TenGi0/7,Gangseo-GDE253-4,20363558880,117149,10581,in,1000000000,1000000000,1000000000,GiGA office Standard,1G


In [18]:
for row in drop_df.iterrows():

    try :
        sql = "INSERT INTO ZABBIXDB.new_db VALUES ('{}', '{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}')".format(row[1][0], row[1][1], row[1][2], row[1][3], row[1][4], row[1][5], row[1][6],
             row[1][7],row[1][8], row[1][9], row[1][10], row[1][11])
        curs.execute(sql)
        conn.commit()
    except pymysql.err.ProgrammingError as e:
        print(sql)
        pass

## 여기까지

###### 문자열 때문에 안들어간 6행 다음에 확인해줄 것

In [152]:
conn = pymysql.connect(host='210.121.218.5', user='zabbix', passwd='zabbix', port=3306, db='test', charset='euckr')
curs = conn.cursor()
sql_command = "SELECT itemid FROM ZABBIXDB.new_db"
curs.execute(sql_command)
fet = curs.fetchall()

In [158]:
li = []

In [ ]:
for row in fet:
    sql = "SELECT DISTINCT a.*, MAX(a.VALUE), n.* FROM alarm_new_70 AS a, new_db AS n WHERE a.itemid = '{}'  AND a.value = (SELECT MAX(value) FROM alarm_new_70 WHERE itemid = '{}') AND n.itemid = '{}'".format(row[0], row[0], row[0])
    curs.execute(sql_command)
    fet = curs.fetchall()
    li.append(fet)

In [132]:
INSERT INTO ZABBIXDB.new_db VALUES ('020002902589','3/2/2','Gonghang-EXA101','23044079280','123110','ifHCInOctets["3/2/2, 10/100/Gig Ethernet SFP, \\ ## T'WayAir, 02000290-2589, 1G ##\\""]','10620','in', '0','0','','GiGA office Standard','1G')
INSERT INTO ZABBIXDB.new_db VALUES ('020002902589','3/2/2','Gonghang-EXA101','23044079280','123236','ifHCOutOctets["3/2/2, 10/100/Gig Ethernet SFP, \\"## T'WayAir, 02000290-2589, 1G ##\\""]','10620','out', '0','0','','GiGA office Standard','1G')
INSERT INTO ZABBIXDB.new_db VALUES ('021400002647','4/1/8, 10/100/Gig Ethernet SFP','Yangjae-EXA178','11512411930','175414','ifHCInOctets["4/1/8, 10/100/Gig Ethernet SFP, \\"## 02140000-2647, prosecutors', 100M, OFD#25-95,96 ##\\""]','10648','in', '100000000','100000000','100000000','','')
INSERT INTO ZABBIXDB.new_db VALUES ('021400002647','4/1/8, 10/100/Gig Ethernet SFP','Yangjae-EXA178','11512411930','175579','ifHCOutOctets["4/1/8, 10/100/Gig Ethernet SFP, \\"## 02140000-2647, prosecutors', 100M, OFD#25-95,96 ##\\""]','10648','out', '100000000','100000000','100000000','','')
INSERT INTO ZABBIXDB.new_db VALUES ('021901052573','3/1/13, 10/100/Gig Ethernet SFP','Geyang-EXA061','24782025460','170139','ifHCInOctets["3/1/13, 10/100/Gig Ethernet SFP, \\"## 02190105-2573, Kyung-In Women's university_BGP AS55626 ##\\""]','10625','in', '1000000000','1000000000','1000000000','','')
INSERT INTO ZABBIXDB.new_db VALUES ('021901052573','3/1/13, 10/100/Gig Ethernet SFP','Geyang-EXA061','24782025460','170271','ifHCOutOctets["3/1/13, 10/100/Gig Ethernet SFP, \\"## 02190105-2573, Kyung-In Women's university_BGP AS55626 ##\\""]','10625','out', '1000000000','1000000000','1000000000','','')
INSERT INTO ZABBIXDB.new_db VALUES ('021901400042','3/2/20, 10/100/Gig Ethernet SFP','Anyang-EXA198','24789212600','195053','ifHCInOctets["3/2/20, 10/100/Gig Ethernet SFP, \\"##CU, It'sPC, 02190140-0042, 500M, Fiber, OFD#17-131,132##\\""]','10635','in', '1000000000','1000000000','1000000000','','')
INSERT INTO ZABBIXDB.new_db VALUES ('021901400042','3/2/20, 10/100/Gig Ethernet SFP','Anyang-EXA198','24789212600','195055','ifHCOutOctets["3/2/20, 10/100/Gig Ethernet SFP, \\"##CU, It'sPC, 02190140-0042, 500M, Fiber, OFD#17-131,132##\\""]','10635','out', '1000000000','1000000000','1000000000','','')

SyntaxError: invalid syntax (Temp/ipykernel_4692/1471980744.py, line 1)

In [189]:
li_df2.to_csv("db_moham.csv")

PermissionError: [Errno 13] Permission denied: 'db_moham.csv'

In [190]:
li_df2.to_csv("db_moham.csv")

In [94]:
df1 = pd.DataFrame(list(dic.items()), columns = ['leased_line_num', 'itemid'])

In [107]:
df1['itemid'][1]

[((166015, 'ifHCInOctets[TenGi0/3]', 10589),
  (166063, 'ifHCOutOctets[TenGi0/3]', 10589))]

In [163]:
li_df

,line_no,interface,hostname,service_id,itemid,key_,hostid,in_out,regexp
0,line_no,interface,hostname,service_id,itemid,key_,hostid,in_out,regexp
1,020000000029,TenGi0/3,Seocho-GDE008,20600900320,166015,ifHCInOctets[TenGi0/3],10589,in,.+(TenGi0/3)[^0-9]
2,020000000029,TenGi0/3,Seocho-GDE008,20600900320,166063,ifHCOutOctets[TenGi0/3],10589,out,.+(TenGi0/3)[^0-9]
3,020000000058,TenGi0/20,Seocho-GDE004,22226449950,166320,ifHCInOctets[TenGi0/20],10733,in,.+(TenGi0/20)[^0-9]
4,020000000058,TenGi0/20,Seocho-GDE004,22226449950,166368,ifHCOutOctets[TenGi0/20],10733,out,.+(TenGi0/20)[^0-9]
...,...,...,...,...,...,...,...,...,...
23294,028884230079,"5/1/8, 10/100/Gig Ethernet SFP",Songdo-EXA020,10196726780,168776,"ifHCInOctets[""ge5/1/8, IP interface, \\""028884...",10622,out,.+(5/1/8)[^0-9]
23295,028884230079,"5/1/8, 10/100/Gig Ethernet SFP",Songdo-EXA020,10196726780,168843,"ifHCInOctets[""5/1/8, 10/100/Gig Ethernet SFP, ...",10622,in,".+(5/1/8, 10/100/Gig Ethernet SFP)[^0-9]"
23296,028884230079,"5/1/8, 10/100/Gig Ethernet SFP",Songdo-EXA020,10196726780,169002,"ifHCOutOctets[""5/1/8, 10/100/Gig Ethernet SFP,...",10622,out,".+(5/1/8, 10/100/Gig Ethernet SFP)[^0-9]"
23297,028884230079,"5/1/8, 10/100/Gig Ethernet SFP",Songdo-EXA020,10196726780,168843,"ifHCInOctets[""5/1/8, 10/100/Gig Ethernet SFP, ...",10622,in,.+(5/1/8)[^0-9]


In [160]:
re = li_df.drop_duplicates()

In [162]:
li_df.to_csv("db_moham.csv")

In [110]:
df1['in'] = df1['itemid'].str.split("),").str[0]
df1['out'] = df1['itemid'].str.split("),").str[1]

error: unbalanced parenthesis at position 0

In [90]:
dic

defaultdict(list,
            {'020000000010': [()],
             '020000000029': [((166015, 'ifHCInOctets[TenGi0/3]', 10589),
               (166063, 'ifHCOutOctets[TenGi0/3]', 10589))],
             '020000000058': [((166320, 'ifHCInOctets[TenGi0/20]', 10733),
               (166368, 'ifHCOutOctets[TenGi0/20]', 10733)),
              ((166320, 'ifHCInOctets[TenGi0/20]', 10733),
               (166368, 'ifHCOutOctets[TenGi0/20]', 10733)),
              ((166320, 'ifHCInOctets[TenGi0/20]', 10733),
               (166368, 'ifHCOutOctets[TenGi0/20]', 10733)),
              ((166320, 'ifHCInOctets[TenGi0/20]', 10733),
               (166368, 'ifHCOutOctets[TenGi0/20]', 10733)),
              ((166320, 'ifHCInOctets[TenGi0/20]', 10733),
               (166368, 'ifHCOutOctets[TenGi0/20]', 10733))],
             '020000000060': [((119482, 'ifHCInOctets[TenGi0/17]', 10722),
               (119530, 'ifHCOutOctets[TenGi0/17]', 10722))],
             '020000010022': [((60585, 'ifHCInOctets[gi1]'

In [87]:
pd.DataFrame(db_moham_row)

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,"(166015, ifHCInOctets[TenGi0/3], 10589)","(166063, ifHCOutOctets[TenGi0/3], 10589)",None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,"(166320, ifHCInOctets[TenGi0/20], 10733)","(166368, ifHCOutOctets[TenGi0/20], 10733)",None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,"(166320, ifHCInOctets[TenGi0/20], 10733)","(166368, ifHCOutOctets[TenGi0/20], 10733)",None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,"(119482, ifHCInOctets[TenGi0/17], 10722)","(119530, ifHCOutOctets[TenGi0/17], 10722)",None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13094,"(122810, ifHCInOctets[""1/2/18, 10/100/Gig Ethe...","(122764, ifHCInOctets[""ge1/2/18, IP interface,...","(122952, ifHCOutOctets[""1/2/18, 10/100/Gig Eth...","(122906, ifHCOutOctets[""ge1/2/18, IP interface...",None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
13095,"(168843, ifHCInOctets[""5/1/8, 10/100/Gig Ether...","(169002, ifHCOutOctets[""5/1/8, 10/100/Gig Ethe...",None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
13096,"(168843, ifHCInOctets[""5/1/8, 10/100/Gig Ether...","(168776, ifHCInOctets[""ge5/1/8, IP interface, ...","(169002, ifHCOutOctets[""5/1/8, 10/100/Gig Ethe...","(168935, ifHCOutOctets[""ge5/1/8, IP interface,...",None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
13097,"(168843, ifHCInOctets[""5/1/8, 10/100/Gig Ether...","(169002, ifHCOutOctets[""5/1/8, 10/100/Gig Ethe...",None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [59]:
20000/60/60


5.555555555555555

In [27]:
globals()["regexp_list_{}".format("Vlan113")]

[]

In [21]:
regexp_list

['.+(vlan1.77)[^0-9]',
 '.+(TenGi0/3)[^0-9]',
 '.+(TenGi0/20)[^0-9]',
 '.+(TenGi0/20)[^0-9]',
 '.+(TenGi0/17)[^0-9]',
 '.+(gi1)[^0-9]',
 '.+(TenGi0/20)[^0-9]',
 '.+(TenGi0/20)[^0-9]',
 '.+(TenGi0/20)[^0-9]',
 '.+(TenGi0/18)[^0-9]',
 '.+(gi17)[^0-9]',
 '.+(gi4)[^0-9]',
 '.+(3/1/3, 10/100/Gig Ethernet SFP)[^0-9]',
 '.+(3/1/3)[^0-9]',
 '.+(2/1/21, 10/100/Gig Ethernet SFP)[^0-9]',
 '.+(2/1/21)[^0-9]',
 '.+(GigabitEthernet3/24)[^0-9]',
 '.+(2/2/9, 10/100/Gig Ethernet SFP)[^0-9]',
 '.+(2/2/9)[^0-9]',
 '.+(TenGi0/33)[^0-9]',
 '.+(1/2/9, 10-Gig Ethernet)[^0-9]',
 '.+(1/2/9)[^0-9]',
 '.+(1/2/7, 10/100/Gig Ethernet SFP)[^0-9]',
 '.+(1/2/7)[^0-9]',
 '.+(1/2/7, 10/100/Gig Ethernet SFP)[^0-9]',
 '.+(1/2/7)[^0-9]',
 '.+(GigabitEthernet4/5)[^0-9]',
 '.+(TenGi0/10)[^0-9]',
 '.+(GigabitEthernet4/16)[^0-9]',
 '.+(TenGi0/7)[^0-9]',
 '.+(TenGi0/7)[^0-9]',
 '.+(TenGi0/7)[^0-9]',
 '.+(GigabitEthernet2/10)[^0-9]',
 '.+(br11)[^0-9]',
 '.+(GigabitEthernet1/14)[^0-9]',
 '.+(br5)[^0-9]',
 '.+(FastEthernet3/48)[^

In [ ]:
sql2 = "SELECT"

In [ ]:
sql2 = "SELECT itemid, key_, hostid FROM ZABBIXDB.items "

In [8]:
df[6]

0                                vlan1.77
1                                TenGi0/3
2                               TenGi0/20
3                               TenGi0/20
4                               TenGi0/17
                       ...               
105033    1/2/18, 10/100/Gig Ethernet SFP
105034                         et-4/0/8.0
105035                         et-5/0/7.0
105036     5/1/8, 10/100/Gig Ethernet SFP
105037     5/1/8, 10/100/Gig Ethernet SFP
Name: 6, Length: 105038, dtype: object

In [145]:
opt = [' 회선정보', ' IP 자원', ' 임계치초과', 'null', ' 중단여부', ' 최대속도', ' 제공속도', ' IP 자원']

In [150]:
count = 1
li = ""
for row in opt:
    if row == "null":
        pass
    else :
        li = li + str(count)
    count += 1
print(li)

1235678
